# Generating Synthetic Data for Retrieval Augmented Generation. 

In this notebook, we'll generate a synthetic dataset for RAG using Wikipedia. We'll build on all documents in the topic San Francisco.

In [9]:
!pip install llama-index wikipedia tqdm python-dotenv spacy

  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 10.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 683.9 kB/s eta 0:00:0036m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.0/133.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 9.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.5/861.5 kB 11.0 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.4 MB/s eta 0:00:00

[notice] A new release of p

## Setup Environment Variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Save Articles from Wikipedia

We'll first download data from Wikipedia pages about popular cities.

In [4]:
popular_cities = ["San Francisco", "Oakland, California", "San Jose, California"]

In [6]:
import os

import wikipedia

# Create a directory to store the content
documents_folder = os.path.join(os.path.abspath(""), ".content")
os.makedirs(documents_folder, exist_ok=True)


# Function to save article content to a file
def save_article_content(city, folder):
    try:
        # Fetching the content of the city's Wikipedia page
        content = wikipedia.page(city).content
        file_path = os.path.join(folder, city.replace(" ", "_") + ".txt")
        with open(file_path, "w") as file:
            file.write(content)
        return file_path
    except Exception as e:
        return str(e)


# Saving the content of each city to the folder
saved_files = [save_article_content(city, documents_folder) for city in popular_cities]

## Generate Synthetic RAG Data From the Documents

In [11]:
# We'll load data about the histories of some cities from Wikipedia.
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.readers import WikipediaReader

documents = WikipediaReader().load_data(pages=[f"History of {x}" for x in popular_cities])
service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=50)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

### Generate RAG Dataset

In [12]:
# generate questions against chunks
from llama_index import ServiceContext
from llama_index.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms import OpenAI

# set context for llm provider
gpt_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4", temperature=0.3))

# instantiate a DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    service_context=gpt_context,
    num_questions_per_chunk=1,  # set the number of questions per nodes
    show_progress=True,
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
import nest_asyncio

nest_asyncio.apply()
rag_dataset = dataset_generator.generate_dataset_from_nodes()
df = rag_dataset.to_pandas()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.25s/it]


In [16]:
df.head()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"Discuss the early history of San Francisco, in...","[The history of the city of San Francisco, Cal...",The earliest evidence of human habitation in S...,ai (gpt-4),ai (gpt-4)
1,Discuss the early European exploration and set...,[== Arrival of Europeans and early settlement ...,The first documented European sighting of San ...,ai (gpt-4),ai (gpt-4)
2,Discuss the impact of the 1848 California gold...,[== 1848 gold rush ==\nThe California gold rus...,The 1848 California gold rush had a significan...,ai (gpt-4),ai (gpt-4)
3,Discuss the transformation of San Francisco in...,[== Paris of the West ==\n\nIt was during the ...,San Francisco began to transform into a major ...,ai (gpt-4),ai (gpt-4)
4,Discuss the corruption and graft trials involv...,[== Corruption and graft trials ==\n\nMayor Eu...,"Mayor Eugene Schmitz, president of the Musicia...",ai (gpt-4),ai (gpt-4)


### Save the Dataset in HuggingFace format.

In [17]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
print(f"Dataset contains {len(dataset)} rows")
dataset.save_to_disk("rag_synth_data.hf")

Dataset contains 47 rows


Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]